In [1]:
import os
os.environ['AOC_SESSION'] = '53616c7465645f5fe53b184f367847aa55a2617e1b05248cac098b2c3dd14dae217169b3ce2e42b10bcc2af6d9b6787a4792a557f9705bb1eba86d761f2407d2'

In [2]:
import aocd
from aocd.models import Puzzle
day = 16
year = 2022
puzzle = Puzzle(year=year, day=day)
# data = aocd.get_data(day=day, year=year)
with open('./data/input_{:02d}'.format(day), 'w') as fh:
    fh.write(puzzle.input_data)

In [108]:
import networkx as nx
import re
from heapq import heappush, heappop, heapify
from itertools import product

In [4]:
test_data = """Valve AA has flow rate=0; tunnels lead to valves DD, II, BB
Valve BB has flow rate=13; tunnels lead to valves CC, AA
Valve CC has flow rate=2; tunnels lead to valves DD, BB
Valve DD has flow rate=20; tunnels lead to valves CC, AA, EE
Valve EE has flow rate=3; tunnels lead to valves FF, DD
Valve FF has flow rate=0; tunnels lead to valves EE, GG
Valve GG has flow rate=0; tunnels lead to valves FF, HH
Valve HH has flow rate=22; tunnel leads to valve GG
Valve II has flow rate=0; tunnels lead to valves AA, JJ
Valve JJ has flow rate=21; tunnel leads to valve II"""

In [5]:
data = puzzle.input_data.splitlines()
data_test = test_data.splitlines()
len(data), data[:10]

(60,
 ['Valve XB has flow rate=0; tunnels lead to valves WZ, LE',
  'Valve BM has flow rate=0; tunnels lead to valves PL, RI',
  'Valve GC has flow rate=0; tunnels lead to valves HN, IT',
  'Valve RM has flow rate=0; tunnels lead to valves ZQ, YL',
  'Valve ZM has flow rate=5; tunnels lead to valves SN, KE, UW, MY, GW',
  'Valve UH has flow rate=0; tunnels lead to valves HM, HN',
  'Valve GW has flow rate=0; tunnels lead to valves LE, ZM',
  'Valve HN has flow rate=19; tunnels lead to valves UW, UH, GL, WZ, GC',
  'Valve VT has flow rate=0; tunnels lead to valves ZD, PE',
  'Valve VI has flow rate=0; tunnels lead to valves JS, AA'])

In [79]:
def parse(data):
    flow_rates = {}
    G = nx.Graph()
    for line in data:
        valve = line.split(' ', 2)[1]
        try:
            targets = line.split(' valves ')[1].split(', ')
        except IndexError:
            targets = line.split(' ')[-1:]
        for target in targets:
            G.add_edge(valve, target)
        fr = int(re.search('\d+', line)[0])
        flow_rates[valve] = fr
    return G, flow_rates

def find_pressure_optimum(flow_rates, distances, max_time=30):
    targets = set([key for key, val in flow_rates.items() if val > 0])
    queue = [[0, 0, 0, 'AA', tuple(['AA'])]]
    finished = []
    
    # print(len(queue))
    while queue:
        # print(queue)
        time_used, pressure, pressure_per_time, pos, visited = heappop(queue)
        if time_used >= max_time:
            heappush(finished, (time_used, pressure, pressure_per_time, pos, visited))
            return finished, -pressure
            
        candidates = (targets - set(visited))
        new_paths = 0
        
        for target in candidates:
            dist = distances[pos][target]
            if (time_used + dist) < max_time:
                heappush(queue, (time_used+dist+1, pressure+(pressure_per_time * (dist+1)), pressure_per_time-flow_rates[target], target, visited+ (target,) ))                    
                new_paths += 1
        if new_paths == 0:
            heappush(queue, (time_used+1, pressure+pressure_per_time, pressure_per_time, pos, visited))
    
    return finished
            

In [82]:
# G, flow_rates = parse(data_test)
G, flow_rates = parse(data)
distances = dict(nx.all_pairs_shortest_path_length(G))

In [468]:
%%time
result, pressure = find_pressure_optimum(flow_rates, distances)
pressure

CPU times: user 1.16 s, sys: 4 ms, total: 1.16 s
Wall time: 1.17 s


2265

In [85]:
puzzle.answer_a = pressure

That's the right answer!  You are one gold star closer to collecting enough star fruit. [Continue to Part Two]


In [87]:
# Part 2

In [723]:
def find_pressure_optimum_B(flow_rates, nbrs, max_time=26):
    max_flow_rate = -sum(flow_rates.values())
    print(max_flow_rate)
    targets = set([key for key, val in flow_rates.items() if val > 0])
    queue = [[0, 0, 0, 0, 'AA', 'AA', set(), [], []]]
    finished = []
    seen = dict()
    
    # print(len(queue))
    while queue:
        # print(queue)
        prio, time_used, pressure, pressure_per_time, posA, posB, vopen, pathA, pathB = heappop(queue)
        lookup_hash = tuple([time_used, *sorted([posA, posB])])

        # vopen = vopen.copy()
        if time_used >= max_time:            
            heappush(finished, (time_used, pressure, pressure_per_time, posA, posB, vopen, pathA, pathB))
            return queue, finished, pathA, pathB, -pressure
        
        pressure += pressure_per_time
        time_used += 1
        prio = max_flow_rate * (max_time - time_used) + pressure
        
        if lookup_hash in seen:
            if pressure_per_time >= seen[lookup_hash]:
                continue
        seen[lookup_hash] = pressure_per_time
        
        if len(finished):
            # print(finished, prio)
            # return
            if finished[0][1] <= prio:
                # print('skipping')
                continue
        
        if pressure_per_time == max_flow_rate:
            heappush(queue, (prio, time_used, pressure, pressure_per_time, posA, posB, vopen, pathA.copy(), pathB.copy()))
            continue                  

        tgtsA = list(nbrs[posA].keys())
        if ((posA not in vopen) and (flow_rates[posA] > 0)) or (len(tgtsA) == 0):
             tgtsA += [posA]
        
        tgtsB = list(nbrs[posB].keys())
        if ((posB not in vopen) and (flow_rates[posB] > 0) and (posA != posB)) or (len(tgtsB) == 0):
             tgtsB += [posB]
        
        for toA, toB in product(tgtsA, tgtsB):
            delta_flow_rate_A = 0
            delta_flow_rate_B = 0
            to_add = set()
            
            if (posA == toA) and (posA not in vopen):
                to_add.add(posA)
                delta_flow_rate_A = -flow_rates[posA]
                
            if (posB == toB) and (posB not in vopen) and (posA != posB):
                to_add.add(posB)
                delta_flow_rate_B = -flow_rates[posB]
                
            heappush(queue, (
                prio, 
                time_used, 
                pressure, 
                pressure_per_time + delta_flow_rate_A + delta_flow_rate_B, 
                toA, 
                toB, 
                vopen.union(to_add), 
                pathA+[(toA, pressure_per_time + delta_flow_rate_A + delta_flow_rate_B, time_used)], 
                pathB+[toB],
            ))            
            
        if len(queue) > 5000000:
            return queue
    
    return finished

In [724]:
G, flow_rates = parse(data_test)
G, flow_rates = parse(data)
distances = dict(nx.all_pairs_shortest_path_length(G))

In [725]:
%%time
queue, finished, pathA, pathB, pressure = find_pressure_optimum_B(flow_rates, dict(G.adjacency()), max_time=26)
pressure

-223
CPU times: user 12.5 s, sys: 11.8 ms, total: 12.6 s
Wall time: 12.6 s


2811

In [719]:
def find_pressure_optimum_C(flow_rates, distances, max_time=26):
    max_flow_rate = -sum(flow_rates.values())
    all_targets = set([key for key, val in flow_rates.items() if val > 0])
    queue = [[0, 0, 0, 0, ('AA', 0), ('AA', 0), set(), set(['AA']), [], []]]
    finished = []
    seen = dict()
    
    # print(len(queue))
    while queue:
        # print(queue)
        prio, time_used, pressure, pressure_per_time, posA, posB, visited, vopened, pathA, pathB = heappop(queue)
        lookup_hash = tuple([time_used, *sorted([posA, posB])])

        # vopen = vopen.copy()
        if time_used >= max_time:            
            heappush(finished, (time_used, pressure, pressure_per_time, posA, posB, visited, vopened, pathA, pathB))
            return queue, finished, pathA, pathB, -pressure
        
        pressure += pressure_per_time
        time_used += 1
        prio = max_flow_rate * (max_time - time_used) + pressure
        
        if lookup_hash in seen:
            if pressure_per_time >= seen[lookup_hash]:
                # print(lookup_hash, seen[lookup_hash], prio)
                # break
                continue
        seen[lookup_hash] = pressure_per_time
        
        if len(finished):
            # print(finished, prio)
            # return
            if finished[0][1] <= prio:
                # print('skipping')
                continue
        
        if pressure_per_time == max_flow_rate:
            heappush(queue, (prio, time_used, pressure, pressure_per_time, posA, posB, visited, vopened, pathA.copy(), pathB.copy()))
            continue                  

        targets_A = []
        targets_B = []
        
        for pos, targets in ((posA, targets_A), (posB, targets_B)):
            _to, time_left = pos
            candidates = (all_targets - set(visited))
            
            if time_left > 0:
                time_left -= 1
                targets.append((_to, time_left))
                if (_to not in vopened) and (time_left == 0):
                    pressure_per_time -= flow_rates[_to]
                    vopened = vopened.union(set([_to]))
            else:                
                # print(flow_rates, _to, pressure_per_time)
                for cnd in candidates:
                    dist = distances[_to][cnd]
                    if (time_used + dist) <= max_time:
                        targets.append((cnd, dist))
            if len(targets) == 0:
                targets.append((_to, 1))
        
        for toA, toB in product(targets_A, targets_B): 
            heappush(queue, (
                prio, 
                time_used, 
                pressure, 
                pressure_per_time, 
                toA, 
                toB, 
                visited.union(set([toA[0], toB[0]])), 
                vopened,
                pathA + [(toA, time_used, pressure, pressure_per_time, vopened.copy())], 
                pathB + [(toB, time_used)]
            ))
                   
        if len(queue) > 5000000:
            return queue
    
    return finished

In [720]:
G, flow_rates = parse(data_test)
G, flow_rates = parse(data)
distances = dict(nx.all_pairs_shortest_path_length(G))

In [721]:
%%time
queue, finished, pathA, pathB, pressure = find_pressure_optimum_C(flow_rates, distances)
pressure

CPU times: user 4.99 s, sys: 39.9 ms, total: 5.03 s
Wall time: 5.04 s


2811

In [722]:
puzzle.answer_b = pressure

That's the right answer!  You are one gold star closer to collecting enough star fruit.You have completed Day 16! You can [Shareon
  Twitter
Mastodon] this victory or [Return to Your Advent Calendar].
